In [2]:
%pip install missingno pandas seaborn numpy

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2f/0e/3b74e8f7c908082793adafb02753477f653ccd7e189f3ba070757d2d0e65/pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/9b/5a/f265a1ba3641d16b5480a217a6aed08cceef09cd173b568cd5351053472a/numpy-1.26.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.26.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/b5/24/aaccf324ce862b

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno

In [4]:
df = pd.read_csv('googleplaystore.csv')
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
5882,A-Z Screen Recorder -,VIDEO_PLAYERS,NaN,17,3.7M,500+,Free,0,Everyone,Video Players & Editors,"July 23, 2018",1.0,4.4 and up
9143,Real Racing 3,FAMILY,4.5,354454,71M,"10,000,000+",Free,0,Everyone,Racing;Action & Adventure,"July 2, 2018",6.4.0,4.1 and up
396,Android Messages,COMMUNICATION,4.2,781810,Varies with device,"100,000,000+",Free,0,Everyone,Communication,"August 1, 2018",Varies with device,Varies with device
1987,Jewels Star: OZ adventure,GAME,4.5,21892,14M,"1,000,000+",Free,0,Everyone,Puzzle,"December 29, 2017",1.3,4.0 and up
2827,Photo Frame,PHOTOGRAPHY,4.5,74476,23M,"10,000,000+",Free,0,Everyone,Photography,"June 19, 2018",1.6,4.0 and up


#### Data cleaning
1. Which of the following column(s) has/have null values?
2. Clean the Rating column and the other columns containing null values

In [5]:
df.loc[df['Rating'] > 5, 'Rating'] = np.nan

In [6]:
# For `Rating`, which is numeric:
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())

In [7]:
# For the other columns
df.dropna(inplace=True)

3. Clean the column Reviews and make it numeric


In [8]:
df.loc[df['Reviews'].str.contains('M'), 'Reviews'] = (pd.to_numeric(
    df.loc[df['Reviews'].str.contains('M'), 'Reviews'].str.replace('M', '')) * 1_000_000).astype('str')

df['Reviews'] = pd.to_numeric(df['Reviews'])

In [10]:
df['Reviews'].head(10)

0       159
1       967
2     87510
3    215644
4       967
5       167
6       178
7     36815
8     13791
9       121
Name: Reviews, dtype: int64

4. How many duplicated apps are there?

In [11]:
df['App'].duplicated(keep=False).sum()

1979

5. Drop duplicated apps keeping the ones with the greatest number of reviews


In [12]:
df_sorted = df.sort_values(by=['App', 'Reviews'])

In [13]:
df_sorted.loc[
    df_sorted['App'].duplicated(keep=False) & ~df_sorted.duplicated(keep=False),
    ['App', 'Reviews']
].head(5)

,App,Reviews
5415,365Scores - Live Scores,666246
3083,365Scores - Live Scores,666521
3953,8 Ball Pool,14184910
1675,8 Ball Pool,14198297
1703,8 Ball Pool,14198602


In [14]:
df_sorted.drop_duplicates(subset=['App'], keep='last', inplace=True)

In [15]:
df = df_sorted

6. Format the Category column

In [16]:
# Replace underscores with whitespaces
df["Category"] = df['Category'].str.replace('_', ' ')
# Capitalize the column
df['Category'] = df['Category'].str.capitalize()

7. Clean and convert the Installs column to numeric type

In [17]:
df['Installs'] = df['Installs'].str.replace('+', '').str.replace(',', '')
df['Installs'] = pd.to_numeric(df['Installs'])

In [18]:
df['Installs']

8884        500
8532    1000000
324       10000
4541      10000
4636      10000
         ...   
6334     100000
4362      10000
2575    1000000
7559      10000
882     1000000
Name: Installs, Length: 9648, dtype: int64

8. Clean and convert the Size column to numeric (representing bytes)



In [19]:
# Varies with device, we set it to 0 and convert it to string
df['Size'] = df['Size'].replace('Varies with device', "0").astype(str)

In [20]:
# Transform M to ~1M bytes and imputing transform
new_value = (pd.to_numeric(
    df.loc[df['Size'].str.contains('M'), 'Size'].str.replace('M', '')
) * (1024 * 1024)).astype(str)
df.loc[df['Size'].str.contains('M'), 'Size'] = new_value

In [21]:
# Same with k
new_value = (pd.to_numeric(
    df.loc[df['Size'].str.contains('k'), 'Size'].str.replace('k', '')
) * 1024).astype(str)
df.loc[df['Size'].str.contains('k'), 'Size'] = new_value

In [22]:
# Get rid of `+` and `,`
df['Size'] = df['Size'].str.replace('+', '')
df['Size'] = df['Size'].str.replace(',', '')

In [23]:
df['Size'] = pd.to_numeric(df['Size'])

In [24]:
#check type
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9648 entries, 8884 to 882
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9648 non-null   object 
 1   Category        9648 non-null   object 
 2   Rating          9648 non-null   float64
 3   Reviews         9648 non-null   int64  
 4   Size            9648 non-null   float64
 5   Installs        9648 non-null   int64  
 6   Type            9648 non-null   object 
 7   Price           9648 non-null   object 
 8   Content Rating  9648 non-null   object 
 9   Genres          9648 non-null   object 
 10  Last Updated    9648 non-null   object 
 11  Current Ver     9648 non-null   object 
 12  Android Ver     9648 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 1.0+ MB


9. Clean and convert the Price column to numeric


In [25]:
df.loc[df['Price'] == 'Free', 'Price'] = "0"
df['Price'] = df['Price'].str.replace('$', '').str.replace(',', '.')
df['Price'] = pd.to_numeric(df['Price'])

10. Paid or free?


In [26]:
df['Distribution'] = df['Price'].apply(lambda x: 'Paid' if x > 0 else 'Free')

In [ ]:
## another way to do it
#df['Distribution'] = 'Free'
#df.loc[df['Price'] > 0, 'Distribution'] = 'Paid'

11. Which app has the most reviews?

In [27]:
df[['App', 'Reviews']].sort_values(by=['Reviews'], ascending=False).head(1)

,App,Reviews
2544,Facebook,78158306


12. What category has the highest number of apps uploaded to the store?

In [28]:
df['Category'].value_counts(ascending = False) # can use head

Category
Family                 1874
Game                    945
Tools                   827
Business                420
Medical                 395
Productivity            374
Personalization         374
Lifestyle               369
Finance                 345
Sports                  325
Communication           315
Health and fitness      288
Photography             281
News and magazines      254
Social                  239
Books and reference     221
Travel and local        219
Shopping                202
Dating                  170
Video players           164
Maps and navigation     131
Food and drink          112
Education               105
Entertainment            86
Auto and vehicles        85
Libraries and demo       83
Weather                  79
House and home           73
Events                   64
Art and design           60
Parenting                60
Comics                   56
Beauty                   53
Name: count, dtype: int64

13. To which category belongs the most expensive app?

In [30]:
df.sort_values(by='Price').tail(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
4367,I'm Rich - Trump Edition,Lifestyle,3.6,275,7654604.8,10000,Paid,400.0,Everyone,Lifestyle,"May 3, 2018",1.0.1,4.1 and up,Paid


14. What's the name of the most expensive game?

In [31]:
df[df['Category'] == 'Game'].sort_values(by='Price').tail(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
4203,The World Ends With You,Game,4.6,4108,13631488.0,10000,Paid,17.99,Everyone 10+,Arcade,"December 14, 2015",1.0.4,4.0 and up,Paid


15. Which is the most popular Finance App?

In [32]:
df[df['Category'] == 'Finance'].sort_values(by='Installs').tail(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
5601,Google Pay,Finance,4.2,348132,0.0,100000000,Free,0.0,Everyone,Finance,"July 26, 2018",2.70.206190089,Varies with device,Free


16. What Teen Game has the most reviews?

In [33]:
df[(df['Category'] == 'Game') & (df['Content Rating'] == 'Teen') ].sort_values(by='Reviews').tail(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
3912,Asphalt 8: Airborne,Game,4.5,8389714,96468992.0,100000000,Free,0.0,Teen,Racing,"July 4, 2018",3.7.1a,4.0.3 and up,Free


17. Which is the free game with the most reviews?


In [34]:
df[(df['Category'] == 'Game') & (df['Distribution'] == 'Free')].sort_values(by='Reviews').tail(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Distribution
1879,Clash of Clans,Game,4.6,44893888,102760448.0,100000000,Free,0.0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,Free


18. How many TB (terabytes) were transferred (overall) for the most popular Lifestyle app?


In [36]:
app = df.loc[df['Category'] == 'Lifestyle'].sort_values(by='Installs', ascending=False).iloc[0]

(app['Installs'] * app['Size']) / (1024 * 1024 * 1024 * 1024)


6484.9853515625